In [3]:
# Project imports #
import pandas as pd
import numpy as np
import psycopg2 as pg2
import matplotlib.pyplot as plt
import json
import csv
import re
from statsmodels.tsa.seasonal import STL
import statsmodels.api as sm
from datetime import datetime
import calendar
from calendar import monthrange

In [ ]:
listy = []
with open('/Users/Kevin/Desktop/project_dta/csv_pd2006_2020/Kommungruppsindelningen.csv', 'r') as f:
    codes = csv.reader(f)
    for row in codes:
        listy.append(row)

for i in listy[1:]:
    i[1] = int(i[1])
    i[-2] = int(i[-2])
    i[-4] = int(i[-4])

print(listy)
with open(f'/Users/Kevin/Desktop/project_dta/csv_pd2006_2020_city/municipality_codes', mode='w', newline="") as file_writer:
        write = csv.writer(file_writer)
        for row in listy:
            write.writerow(row)

In [ ]:
# Full outer join #
cur.execute("""
    CREATE TABLE historic_ads_extension_1 AS 
    (SELECT ad_id, headline, number_of_vacancies, publication_date, application_deadline, historic_ads_muni.municipality_code, municipality_codes.municipality_name, municipality_codes.municipality_group_code, municipality_codes.municipality_group_name, municipality_codes.county_code, municipality_codes.county_name
    FROM historic_ads_muni
    FULL OUTER JOIN municipality_codes
    ON historic_ads_muni.municipality_code = municipality_codes.municipality_code);
    ;""")
conn.commit()

In [ ]:
# Left join #
cur.execute("""
    CREATE TABLE historic_ads_left AS
    (SELECT m.number_of_vacancies, k.municipality_name 
    FROM historic_ads_muni m 
    LEFT JOIN municipality_codes k ON k.municipality_code = m.municipality_code);
    ;""")
conn.commit()

### Anomaly detection - Daily index

In [ ]:
resid_mu = res_resid.mean()
resid_dev = res_resid.std()
lower = resid_mu - 3*resid_dev
upper = resid_mu + 3*resid_dev
plt.figure(figsize=(10,4))
plt.plot(res_resid)
plt.fill_between([datetime(2006,1,1), datetime(2020,12,31)], lower, upper, color='g', alpha=0.25, linestyle='--', linewidth=2)
plt.xticks(ticks=date_index, labels=date_range)
plt.xlim(datetime(2005,10,1), datetime(2021,3,1))
plt.show()

### Connecting ssyk_codes

In [4]:
x = pd.read_csv('/Users/Kevin/Desktop/project_dta/extension_2/ssyk_levels/ssyk_2.csv', names=['ssyk_2_code', 'ssyk_2_desc'], sep=';')
y = pd.read_excel('/Users/Kevin/Desktop/project_dta/extension_2/ssyk_levels/ssyk_3_4.xls', names=['ssyk_4_code', 'ssyk_4_desc'], header=None)
z = pd.read_excel('/Users/Kevin/Desktop/project_dta/extension_2/ssyk_levels/ssyk_1.xls', names=['ssyk_1_code', 'ssyk_1_desc'], header=None)

In [5]:
y['ssyk_2_code'] = y['ssyk_4_code'].apply(lambda x:str(x)).apply(lambda x:int(x[0:2]))
y = y.reindex(['ssyk_2_code','ssyk_4_code','ssyk_4_desc'], axis=1)
# Line below need adjusting depending on loaded data #
y.iloc[0:3,0] = y.iloc[0:3,0].apply(lambda x:str(x)).apply(lambda x:int(x[0]))

In [6]:
y = y.merge(x, on='ssyk_2_code')
y = y.reindex(['ssyk_2_code','ssyk_2_desc', 'ssyk_4_code', 'ssyk_4_desc'], axis=1)
y.to_csv('/Users/Kevin/Desktop/project_dta/extension_2/ssyk_levels/ssyk_merged.csv', sep=';', index=False, header=False)

In [7]:
merge = pd.read_csv('/Users/Kevin/Desktop/project_dta/extension_2/ssyk_levels/ssyk_merged.csv', sep=';', names=['ssyk_2_code', 'ssyk_2_desc','ssyk_4_code','ssyk_4_desc'], header=None)

In [8]:
#y['ssyk_2_code'] = y['ssyk_4_code'].apply(lambda x:str(x)).apply(lambda x:int(x[0:2]))
merge.iloc[:3,0] = merge.iloc[:3,0].apply(lambda x:0)
pd.options.display.max_rows = 999
merge['ssyk_1_code'] = merge['ssyk_2_code'].apply(lambda x:str(x)).apply(lambda x:int(x[0:1]))
merge = merge.reindex(['ssyk_1_code', 'ssyk_2_code','ssyk_2_desc', 'ssyk_4_code', 'ssyk_4_desc'], axis=1)
merge = z.merge(merge, on='ssyk_1_code')
merge.to_csv('/Users/Kevin/Desktop/project_dta/extension_2/ssyk_levels/ssyk_merged.csv', sep=';', index=False, header=False)

In [12]:
# Removal of problematci withspace #
merge['ssyk_1_desc'] = merge['ssyk_1_desc'].apply(lambda x:x.strip())
merge['ssyk_2_desc'] = merge['ssyk_2_desc'].apply(lambda x:x.strip())
merge['ssyk_4_desc'] = merge['ssyk_4_desc'].apply(lambda x:x.strip())

In [11]:
merge.to_csv('/Users/Kevin/Desktop/project_dta/extension_2/ssyk_levels/ssyk_merged.csv', sep=';', index=False, header=False)

### Extraction of SSYK

In [ ]:
ssyk_1 = []
ssyk_1.append(['Date', 'Chefsyrken', 'Militära yrken', 'Service-, omsorgs- och försäljningsyrken', 'Yrken inom administration och kundtjänst'
               ,'Yrken inom byggverksamhet och tillverkning', 'Yrken inom lantbruk, trädgård, skogsbruk och fiske'
               , 'Yrken inom maskinell tillverkning och transport m.m.', 'Yrken med krav på fördjupad högskolekompetens'
               , 'Yrken med krav på kortare utbildning eller introduktion'])

# Analysis of labor demand by occupational group #
for year in range(2008, 2021):
    for month in range(1,13):
        for day in range(1, calendar.monthrange(year, month)[1] + 1):
            cur.execute(f"""
            SELECT SUM(number_of_vacancies), ssyk_desc_1 FROM historic_ads_2 
            WHERE '{year}-{month}-{day}' >= publication_date AND '{year}-{month}-{day}' <= application_deadline
            GROUP BY ssyk_desc_1
            ;""")
            # The fetch contains at least 9 values, when len > 9 the military professions are included #
            occupation_group = cur.fetchall()
            if len(occupation_group) < 10:
                ssyk_1.append([f'{year}-{month}-{day}', occupation_group[0][0], np.nan, occupation_group[1][0], occupation_group[2][0]
                               , occupation_group[3][0], occupation_group[4][0], occupation_group[5][0], occupation_group[6][0] 
                               , occupation_group[7][0], occupation_group[8][0]])
            # If military profession is in, its in index 1 #
            else:
                ssyk_1.append([f'{year}-{month}-{day}', occupation_group[0][0], occupation_group[1][0],occupation_group[2][0]
                               , occupation_group[3][0], occupation_group[4][0], occupation_group[5][0], occupation_group[6][0] 
                               , occupation_group[7][0], occupation_group[8][0], occupation_group[9][0]])